YBIGTA 10기 노혜미

# tensor board

- 텐서 플로우의 구조, 텐서 값의 변화 같은 것들을 시각적으로 보여주는 기능을 한다.

소스코드 출처
: https://github.com/hunkim/DeepLearningZeroToAll/

multivariable linear regression의 코드를 그대로 가져왔다.

In [1]:
import tensorflow as tf

In [2]:
x_data = [[73., 80., 75.],
          [93., 88., 93.],
          [89., 91., 90.],
          [96., 98., 100.],
          [73., 66., 70.]] # rank2, 5X3 matrix
y_data = [[152.],
          [185.],
          [180.],
          [196.],
          [142.]] # rank2, 5X1 matrix
X = tf.placeholder(tf.float32, shape=[None, 3], name = 'X') 
Y = tf.placeholder(tf.float32, shape=[None, 1], name = 'Y') 

In [3]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([3, 1]), name='weight_1')
    b1 = tf.Variable(tf.random_normal([1]), name='bias_1')
    layer1 = tf.matmul(X, W1)+b1 # 5x3 3x1 = 5x1
    
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)
    
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([1, 1]), name='weight_2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias_2')
    hypothesis = tf.matmul(layer1, W2)+b2
    
    w2_hist = tf.summary.histogram("weights_2", W2)
    b2_hist = tf.summary.histogram("biases_2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [4]:
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.square(hypothesis-Y))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [5]:
with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    # FileWriter의 인자로 경로를 주면 소스코드가 쓰이는 경로에 해당 파일이 만들어지고 log가 기록된다.
    writer = tf.summary.FileWriter("./board/multivariable_linear_regression")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step) 

        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

0 34871.2 [array([[-1.27761829],
       [-0.33085072],
       [ 0.42682689]], dtype=float32), array([[ 0.14722224]], dtype=float32)]
5000 0.285806 [array([[-1.81341398],
       [-0.90011162],
       [-0.21268174]], dtype=float32), array([[-0.69263703]], dtype=float32)]
10000 0.278761 [array([[-1.69697487],
       [-0.84367871],
       [-0.54235804]], dtype=float32), array([[-0.6530109]], dtype=float32)]


참고: http://tensorflowstepbystep.tistory.com/4

### 코드 실행 후

1. anaconda prompt를 관리자 권한으로 실행한다.(anaconda prompt를 우클릭하면 관리자 권한으로 실행할 수 있다.)
2. activate tensorflow env입력
3. cd 위에서 만든 파일이 들어있는 경로 입력
- ex) cd C:\Users\nhm\ybigta\DeepLearningZeroToAll-b00a345e02ca751bdfcca73787aa5fd97e56bd8b
4. 해당 경로로 이동된 걸 확인하고 tensorboard --logdir=log저장 directory
- ex) tensorboard --logdir=board/multivariable_linear_regression
5. Starting TensorBoard b'54' at http://DESKTOP-16K2P34:6006 이러한 문장이 나오면 성공! 해당 웹 주소를 그대로 복사해서 들어가면 된다.

### 기능

![graph](http://i.imgur.com/21cLV8m.png)
- 전체적인 구조가 어떤지 알 수 있다.

![layer](http://i.imgur.com/pHl6H2d.png)
- layer가 어떻게 구성되었는지 또한 볼 수 있다.
- 심지어 weight나 bias에 어떤 optimize를 쓰는 지도 나온다.

![graph](http://i.imgur.com/XORLdY2.png)
- 그래프의 x축은 step이다.
- bias와 weight가 시간이 지날 수록 어떻게 변화했는지 나타내준다.

![histogram](http://i.imgur.com/Yq7UpaE.png)
- 입체적으로 보여준다.
- 세로가 step이다.
- 위의 선형 그래프를 왼쪽으로 돌렸다고 생각하면 될 것 같다.

![cost](http://i.imgur.com/OPRu0Zf.png)
- 일일이 cost를 print할 필요 없이 그래프로 한 눈에 파악할 수 있다.